This notebook contains code for processing Global Surface Water data from raster format into water extent polygons and then computing time series of area within each polygon. It also retrieves meteorological forcings for the area within the polygons. The basic input require for this script is a series of latitude/longitude coordinates that delineate a rectangular area of interest. 

Most of the computation will be done using the `ee` API for Google Earth Engine.

In [6]:
%matplotlib inline
import ee
import json
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import shape

Before starting, we'll specify a prefix for all of our files that will be downloaded. We'll also make an empty dict which will later be used to hold Earth Engine API request objects.

In [ ]:
run_name = 'test_2019_06_26_'
tasks = {}

Prior to running the next code cell, the Earth Engine API must be configured on your computer. The instructions for doing so can be found [here](https://developers.google.com/earth-engine/python_install_manual).

In [7]:
ee.Initialize()

Next, we'll define several functions that will be used to process the water data. Suppose that we have a `FeatureCollection` indicating where water can be found. `single_time_areas` is used to compute a per-water polygon mean value for a single timestep and then add that mean value as metadata to the basin features.

`multiple_time_areas` then maps the above function over an entire `ImageCollection` such that we are eventually given metadata which contains time series for each polygon.

In [8]:
def single_time_areas(image,basins,reducer=ee.Reducer.mean,propname='mean',aggregate=True):
    features   = image.reduceRegions(basins,reducer=reducer())
    if aggregate:
        return features.aggregate_array(propname)
    else:
        return features

def multiple_time_areas(image_collection,basins,reducer=ee.Reducer.mean,propname='mean',aggregate=True):
    new_images = image_collection.map(lambda x: ee.Image().set('zonal_statistics',single_time_areas(x,
                                                                                                    basins,
                                                                                                    reducer=reducer,
                                                                                                    propname=propname,
                                                                                                    aggregate=aggregate)))
    return new_images

At this stage, we indicate our area of interest and also see how big it is.

In [9]:
upper  = 48.9
lower  = 48.5
left   = -99.0
right  = -98.5


bounds = ee.Geometry.Polygon([[[left, lower], [left, upper], 
                               [ right,upper], [right,lower], 
                               [left,lower]]])

area_m2 = bounds.area().getInfo()
area_km2 = area_m2 / 1000000
print('Areal extent is {0} square kilometers.'.format(area_km2))

Areal extent is 1632.0864555544824 square kilometers.


We also need a `FeatureCollection` indicating the maximum extent of each water body. To do this, we get the `max_extent` layer which has a 1 for water ever occurring and 0 otherwise. However, if we try to apply raster-to-vector conversion on this, Earth Engine will create separate polygons for 1 and 0 and the 0-valued polygons will lead to errors. Instead, we want to mask out the `max_extent` layer such that it has only 1 and no-data values.

In [278]:
max_extent = ee.Image('JRC/GSW1_0/GlobalSurfaceWater').select('max_extent').clip(bounds)
max_extent = max_extent.updateMask(max_extent)


In case we want to filter out water polygons which are too large or too small, we can do this with the code cell below.

In [280]:
min_pixels = 5
max_pixels = 10000

basins = max_extent.reduceToVectors(maxPixels=1000000000)
basins = basins.map(area_perimeter_ratio)
basins = basins.filterMetadata('count','greater_than',min_pixels).filterMetadata('count','less_than',max_pixels)

Some of the basins are too small to have non-null overlap with PRISM as PRISM cells are much larger than individual GSW pixels. To solve this issue, we buffer all of them by 500 meters.

In [281]:
buffered_basins = basins.map(lambda x: x.buffer(500))

Let's see how many water polygons or basins we can identify:

In [282]:
tasks['basins'] = ee.batch.Export.table.toDrive(basins, description=run_name + 'basins', folder='Research')
n_basins = basins.size().getInfo()
print(n_basins)

4233


 Now that we have delineated individual water bodies, we want to calculate time series of inundation within each one. This code cell sends a request to Earth Engine to export tables with time series of monthly inundation fraction for each basin.

In [284]:
WATER_PRESENT_VALUE = 2
WATER_ABSENT_VALUES = 1
DATA_MISSING_VALUE  = 0

gsw_monthly       = ee.ImageCollection('JRC/GSW1_0/MonthlyHistory')
gsw_water_present = gsw_monthly.map(lambda x: x.eq(WATER_PRESENT_VALUE))
gsw_missing_data  = gsw_monthly.map(lambda x: x.eq(DATA_MISSING_VALUE))

output_water        = multiple_time_areas(gsw_water_present,basins)
output_missing_data = multiple_time_areas(gsw_missing_data,basins)

tasks['water'] = ee.batch.Export.table.toDrive(output_water, description=run_name + 'water_areas', folder='Research')
tasks['missing_data'] = ee.batch.Export.table.toDrive(output_missing_data, description=run_name + 'missing_data', folder='Research')


We will also do the same thing with PRISM forcings. Here, I'm only looking at precipitation, temperature and vapor pressure deficit, though other variables exist as well.

In [285]:
start_date = '1984-03-01'
end_date   = '2015-10-31'
band_names = ['ppt', 'tmean', 'vpdmax']
prism = ee.ImageCollection('OREGONSTATE/PRISM/AN81m').filterDate(start_date,end_date)
prism = prism.map(lambda x: x.clip(bounds))

for band in band_names:
    out = multiple_time_areas(prism.select(band),buffered_basins)
    tasks[band] = ee.batch.Export.table.toDrive(out, description= run_name + band, folder='Research')

If we are interested in obtaining land use / land change statistics, the NASS cropland data layer is an invaluable source of data. I will note that this is available at a different temporal resolution and so cannot be aligned with PRISM and GSW.

In [286]:
CDL_WATER_CODE_1 = 111
CDL_WATER_CODE_2 = 83

cdl = ee.ImageCollection('USDA/NASS/CDL').select('cropland')

# Since we want the dominant land use around the water body, we need to make sure we aren't counting water
# as one of the possible land uses.
cdl_no_water = cdl.map(lambda x: x.updateMask(x.neq(CDL_WATER_CODE_1).And(x.neq(CDL_WATER_CODE_2))))
cdl_out = multiple_time_areas(cdl_no_water,buffered_basins,reducer=ee.Reducer.mode,propname='mode')

tasks['cdl'] = ee.batch.Export.table.toDrive(cdl_out, description=run_name + 'cdl', folder='Research')


The previous code cells added API request objects to `tasks` but did not start them. Here, we start all of these requests at the same time.

In [287]:
for variable in tasks.keys():
    print(variable)
    tasks[variable].start()

basins
water
missing_data
ppt
tmean
vpdmax
cdl


These tasks may take awhile to run. You can repeatedly run the cell below to check the status of the tasks. The tasks should say `COMPLETED` when ready.

In [306]:
for variable in tasks.keys():
    print('Variable: {0}, status: {1}'.format(variable,tasks[variable].status()['state']))

Variable: basins, status: COMPLETED
Variable: water, status: COMPLETED
Variable: missing_data, status: COMPLETED
Variable: ppt, status: RUNNING
Variable: tmean, status: RUNNING
Variable: vpdmax, status: RUNNING
Variable: cdl, status: COMPLETED


We need to get the data off of our Google Drive and into local storage. I personally use Insync, a 3rd party client which syncs Google Drive to a folder on my computer. You could also drag-and-drop the files manually from Google Drive. Here, I just indicate where Insync has saved my files.

In [10]:
outputs_directory = '/home/ckrapu/ckrapu@gmail.com/Research/'

The next few code cells take the GEE export files and turn them into dataframes. There are numerous details from `pandas` and `geopandas` here and I'm not going to explain them.

In [11]:
dataframes = {}
filename = run_name + 'basins.csv'
basin_gdf = pd.read_csv(outputs_directory + filename)
as_shape = [shape(json.loads(feature)) for feature in basin_gdf['.geo'].values]
basin_gdf['.geo'] = as_shape
basin_gdf = basin_gdf.set_geometry('.geo')
basin_gdf.crs={'init':'epsg:4326'}
dataframes['basins'] = basin_gdf

These code cells take the CSV-formatted files generated by Google Earth Engine and parses them. It takes a string-values column and turns that into a 2D array of values over basins and timesteps.

In [12]:
index_formats = {'water_areas':'yyyy_mm', 
                 'missing_data':'yyyy_mm',
                 'ppt': 'yyyymm',
                 'tmean': 'yyyymm',
                 'vpdmax':'yyyymm'}
                 #'cdl': 'yyyy'}

In [13]:
variables = index_formats.keys()

for variable in variables:
    filename = run_name + variable + '.csv'
    index_format = index_formats[variable]
    
    # Load in CSV generated by GEE and use the 
    # system:index column to generate a Pandas Index
    df = pd.read_csv(outputs_directory + filename)
    series = df.set_index(df['system:index'])['zonal_statistics']
    n_basins = len([float(x) for x in series.iloc[0].replace('[','').replace(']','').split(',')])
      
     # Handle an edge case with the CDL indexing
    if variable == 'cdl':
        series = series.drop(labels=['2005b', '2007b'])
    
    n_timesteps = series.shape[0]
    data = np.zeros([n_timesteps,n_basins])
    
    # For each of the rows, take the CSV's data and place it into the dataframe
    for i in range(n_timesteps):
        numeric_values = [float(x) for x in series.iloc[i].replace('[','').replace(']','').split(',')]
        data[i,:] = numeric_values
        
    old_index = series.index 
    years  = series.index.map(lambda x: int(str(x)[0:4]))
    
    # The format of system:index depends on the dataset used in GEE
    if index_format == 'yyyymm': 
        months = series.index.map(lambda x: int(str(x)[4::]))
    elif index_format == 'yyyy_mm':
        months = series.index.map(lambda x: int(str(x)[5::]))
    elif index_format == 'yyyy':
        months = [1] * years.shape[0]
    
    dates = [pd.Timestamp(year=years[i],month=months[i],day=1) for i in range(len(years))]
    datetime_index = pd.to_datetime(dates)
    dataframes[variable] = pd.DataFrame(index=datetime_index,data=data)


In [17]:
is_missing = dataframes['missing_data'] > 0.
dataframes['water_masked']  = pd.DataFrame(index=dataframes['water_areas'].index,data=dataframes['water_areas'].values)
dataframes['water_masked'][is_missing] = np.nan
areas = dataframes['water_masked'].values * dataframes['basins']['count'].values[np.newaxis,:]


At the end of all of this, you can save the tables in `dataframes` to disk. I'd recommend using `pandas` function `to_pickle` or `to_csv`.